<a href="https://colab.research.google.com/github/sahilbaishya/HandsOnML/blob/main/Training_Deep_Neural_Networks_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Xavier Garlot and He Initialization

In [2]:
[name for name in dir(keras.initializers) if not name.startswith("_")]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'HeNormal',
 'HeUniform',
 'Identity',
 'Initializer',
 'LecunNormal',
 'LecunUniform',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'random_normal',
 'random_uniform',
 'serialize',
 'truncated_normal',
 'variance_scaling',
 'zeros']

# Nonsaturating Activation Functions

In [3]:
[m for m in dir(keras.activations) if not m.startswith("_")]

['deserialize',
 'elu',
 'exponential',
 'gelu',
 'get',
 'hard_sigmoid',
 'linear',
 'relu',
 'selu',
 'serialize',
 'sigmoid',
 'softmax',
 'softplus',
 'softsign',
 'swish',
 'tanh']

In [4]:
[m for m in dir(keras.layers) if "relu" in m.lower()]

['LeakyReLU', 'PReLU', 'ReLU', 'ThresholdedReLU']

## Neural Network on Fashion MNIST with Leaky Relu

In [5]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [6]:
X_train_full = X_train_full/255.0
X_test = X_test/255.0

In [7]:
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [8]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [9]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [10]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 14s 4ms/step - loss: 1.2828 - accuracy: 0.6074 - val_loss: 0.8695 - val_accuracy: 0.7246
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7816 - accuracy: 0.7472 - val_loss: 0.7014 - val_accuracy: 0.7788
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6702 - accuracy: 0.7831 - val_loss: 0.6357 - val_accuracy: 0.7948
Epoch 4/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.6123 - accuracy: 0.8002 - val_loss: 0.5830 - val_accuracy: 0.8136
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5752 - accuracy: 0.8105 - val_loss: 0.5525 - val_accuracy: 0.8228
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5489 - accuracy: 0.8163 - val_loss: 0.5301 - val_accuracy: 0.8272
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5291 - accuracy: 0.8218 - val_loss: 0.5119 - val_accuracy

## Using PReLU

In [11]:
tf.random.set_seed(42)
np.random.seed(42)

model = tf.keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    keras.layers.Dense(300, kernel_initializer = 'he_normal'),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer = 'he_normal'),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation= 'softmax')

])

In [12]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = keras.optimizers.SGD(learning_rate = 1e-3),
              metrics = ['accuracy'])

In [13]:
history = model.fit(X_train, y_train, epochs = 10, validation_data = (X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 1.3677 - accuracy: 0.5896 - val_loss: 0.8986 - val_accuracy: 0.7308
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.8025 - accuracy: 0.7512 - val_loss: 0.7078 - val_accuracy: 0.7834
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6807 - accuracy: 0.7858 - val_loss: 0.6367 - val_accuracy: 0.7992
Epoch 4/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.6180 - accuracy: 0.8035 - val_loss: 0.5799 - val_accuracy: 0.8188
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5775 - accuracy: 0.8122 - val_loss: 0.5469 - val_accuracy: 0.8296
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5486 - accuracy: 0.8204 - val_loss: 0.5236 - val_accuracy: 0.8338
Epoch 7/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5267 - accuracy: 0.8255 - val_loss: 0.5038 - val_accuracy:

## Using SELU

In [14]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[28, 28]))
model.add(tf.keras.layers.Dense(300, activation = 'selu', kernel_initializer = 'lecun_normal'))
for i in range (99):
  model.add(tf.keras.layers.Dense(100, activation = 'selu', kernel_initializer = 'lecun_normal'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

In [15]:
model.compile(
    optimizer = tf.keras.optimizers.SGD(learning_rate = 1e-3),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [16]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [17]:
history = model.fit(X_train_scaled, y_train, epochs = 10, validation_data = (X_valid_scaled, y_valid))

Epoch 1/10
1719/1719 [==============================] - 54s 23ms/step - loss: 1.1282 - accuracy: 0.5762 - val_loss: 0.7678 - val_accuracy: 0.7190
Epoch 2/10
1719/1719 [==============================] - 40s 23ms/step - loss: 0.7211 - accuracy: 0.7327 - val_loss: 0.7078 - val_accuracy: 0.7582
Epoch 3/10
1719/1719 [==============================] - 41s 24ms/step - loss: 0.6318 - accuracy: 0.7691 - val_loss: 0.5926 - val_accuracy: 0.7856
Epoch 4/10
1719/1719 [==============================] - 40s 23ms/step - loss: 0.6443 - accuracy: 0.7704 - val_loss: 0.6406 - val_accuracy: 0.7824
Epoch 5/10
1719/1719 [==============================] - 41s 24ms/step - loss: 0.5649 - accuracy: 0.7994 - val_loss: 0.5197 - val_accuracy: 0.8228
Epoch 6/10
1719/1719 [==============================] - 41s 24ms/step - loss: 0.4980 - accuracy: 0.8280 - val_loss: 0.5382 - val_accuracy: 0.8028
Epoch 7/10
1719/1719 [==============================] - 40s 23ms/step - loss: 0.4777 - accuracy: 0.8351 - val_loss: 0.4508 -

## Using ReLU

In [18]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape = [28, 28]))
model.add(tf.keras.layers.Dense(300, activation = 'relu', kernel_initializer='he_normal'))
for i in range (99):
  model.add(tf.keras.layers.Dense(100, activation = 'relu', kernel_initializer = 'he_normal'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

In [19]:
model.compile(
    optimizer = tf.keras.optimizers.SGD(learning_rate = 1e-3),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [20]:
history = model.fit(X_train_scaled, y_train, epochs = 10, validation_data = (X_valid_scaled, y_valid))

Epoch 1/10
1719/1719 [==============================] - 56s 25ms/step - loss: 1.9071 - accuracy: 0.2700 - val_loss: 1.4582 - val_accuracy: 0.4296
Epoch 2/10
1719/1719 [==============================] - 41s 24ms/step - loss: 1.2752 - accuracy: 0.4612 - val_loss: 0.9763 - val_accuracy: 0.6244
Epoch 3/10
1719/1719 [==============================] - 41s 24ms/step - loss: 0.9463 - accuracy: 0.6170 - val_loss: 0.8247 - val_accuracy: 0.6808
Epoch 4/10
1719/1719 [==============================] - 39s 23ms/step - loss: 0.7916 - accuracy: 0.6798 - val_loss: 0.8840 - val_accuracy: 0.6512
Epoch 5/10
1719/1719 [==============================] - 39s 23ms/step - loss: 0.7049 - accuracy: 0.7097 - val_loss: 0.6747 - val_accuracy: 0.7196
Epoch 6/10
1719/1719 [==============================] - 39s 23ms/step - loss: 0.6526 - accuracy: 0.7323 - val_loss: 0.6387 - val_accuracy: 0.7476
Epoch 7/10
1719/1719 [==============================] - 39s 23ms/step - loss: 0.6295 - accuracy: 0.7477 - val_loss: 0.6002 -

# Batch Normalization

In [21]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [22]:
X_train_full = X_train_full/255.0
X_test = X_test/255.0

In [23]:
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [24]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = [28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation = 'softmax')
])

In [25]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_208 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_209 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [26]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [27]:
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = tf.keras.optimizers.SGD(learning_rate = 1e-3),
    metrics = ['accuracy']
)

In [28]:
history = model.fit(X_train, y_train, epochs = 10, validation_data = (X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 10s 5ms/step - loss: 0.8425 - accuracy: 0.7189 - val_loss: 0.5427 - val_accuracy: 0.8190
Epoch 2/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5653 - accuracy: 0.8036 - val_loss: 0.4616 - val_accuracy: 0.8466
Epoch 3/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5111 - accuracy: 0.8221 - val_loss: 0.4283 - val_accuracy: 0.8598
Epoch 4/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4735 - accuracy: 0.8339 - val_loss: 0.4089 - val_accuracy: 0.8630
Epoch 5/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4510 - accuracy: 0.8420 - val_loss: 0.3938 - val_accuracy: 0.8700
Epoch 6/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4345 - accuracy: 0.8466 - val_loss: 0.3835 - val_accuracy: 0.8740
Epoch 7/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4205 - accuracy: 0.8522 - val_loss: 0.3734 - val_accuracy

## Applying Batch Normalization Before Activation

In [29]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])


In [30]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])


In [31]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))


Epoch 1/10
1719/1719 [==============================] - 9s 5ms/step - loss: 1.0224 - accuracy: 0.6838 - val_loss: 0.6719 - val_accuracy: 0.7854
Epoch 2/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.6726 - accuracy: 0.7823 - val_loss: 0.5563 - val_accuracy: 0.8160
Epoch 3/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5942 - accuracy: 0.8028 - val_loss: 0.5036 - val_accuracy: 0.8314
Epoch 4/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5459 - accuracy: 0.8175 - val_loss: 0.4711 - val_accuracy: 0.8366
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5126 - accuracy: 0.8258 - val_loss: 0.4492 - val_accuracy: 0.8432
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4910 - accuracy: 0.8310 - val_loss: 0.4314 - val_accuracy: 0.8480
Epoch 7/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4740 - accuracy: 0.8373 - val_loss: 0.4181 - val_accuracy:

# Gradient Clipping

In [32]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)

In [34]:
optimizer = keras.optimizers.SGD(clipnorm=1.0)

# Reusing Pretrained Layers

In [35]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [36]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [37]:
X_train_A.shape

(43986, 28, 28)

In [38]:
X_train_B.shape

(200, 28, 28)

In [39]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [40]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [41]:
tf.random.set_seed(42)
np.random.seed(42)

In [42]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [43]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [44]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.6152 - accuracy: 0.7943 - val_loss: 0.3984 - val_accuracy: 0.8582
Epoch 2/20
1375/1375 [==============================] - 6s 5ms/step - loss: 0.3658 - accuracy: 0.8756 - val_loss: 0.3298 - val_accuracy: 0.8909
Epoch 3/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3249 - accuracy: 0.8884 - val_loss: 0.2994 - val_accuracy: 0.9026
Epoch 4/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.3043 - accuracy: 0.8962 - val_loss: 0.2846 - val_accuracy: 0.9073
Epoch 5/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2908 - accuracy: 0.9002 - val_loss: 0.2737 - val_accuracy: 0.9088
Epoch 6/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2809 - accuracy: 0.9040 - val_loss: 0.2673 - val_accuracy: 0.9131
Epoch 7/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2724 - accuracy: 0.9074 - val_loss: 0.2640 - val_accuracy:

In [45]:
model_A.save("my_model_A.h5")

In [46]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [47]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [49]:
model_B.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 784)               0         
                                                                 
 dense_220 (Dense)           (None, 300)               235500    
                                                                 
 dense_221 (Dense)           (None, 100)               30100     
                                                                 
 dense_222 (Dense)           (None, 50)                5050      
                                                                 
 dense_223 (Dense)           (None, 50)                2550      
                                                                 
 dense_224 (Dense)           (None, 50)                2550      
                                                                 
 dense_225 (Dense)           (None, 1)                

In [48]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 2s 158ms/step - loss: 0.7857 - accuracy: 0.5550 - val_loss: 0.6404 - val_accuracy: 0.6166
Epoch 2/20
7/7 [==============================] - 0s 18ms/step - loss: 0.6126 - accuracy: 0.6400 - val_loss: 0.5111 - val_accuracy: 0.7667
Epoch 3/20
7/7 [==============================] - 0s 18ms/step - loss: 0.4899 - accuracy: 0.7750 - val_loss: 0.4268 - val_accuracy: 0.8631
Epoch 4/20
7/7 [==============================] - 0s 18ms/step - loss: 0.4073 - accuracy: 0.8400 - val_loss: 0.3648 - val_accuracy: 0.9067
Epoch 5/20
7/7 [==============================] - 0s 18ms/step - loss: 0.3455 - accuracy: 0.9000 - val_loss: 0.3141 - val_accuracy: 0.9412
Epoch 6/20
7/7 [==============================] - 0s 31ms/step - loss: 0.2939 - accuracy: 0.9400 - val_loss: 0.2777 - val_accuracy: 0.9473
Epoch 7/20
7/7 [==============================] - 0s 31ms/step - loss: 0.2570 - accuracy: 0.9500 - val_loss: 0.2488 - val_accuracy: 0.9554
Epoch 8/20
7/7 [==========

In [50]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [51]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [52]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

In [53]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 42ms/step - loss: 0.5763 - accuracy: 0.6500 - val_loss: 0.5649 - val_accuracy: 0.6826
Epoch 2/4
7/7 [==============================] - 0s 18ms/step - loss: 0.5448 - accuracy: 0.6600 - val_loss: 0.5342 - val_accuracy: 0.7028
Epoch 3/4
7/7 [==============================] - 0s 17ms/step - loss: 0.5140 - accuracy: 0.6850 - val_loss: 0.5079 - val_accuracy: 0.7312
Epoch 4/4
7/7 [==============================] - 0s 16ms/step - loss: 0.4877 - accuracy: 0.7150 - val_loss: 0.4831 - val_accuracy: 0.7596
Epoch 1/16
7/7 [==============================] - 1s 57ms/step - loss: 0.3905 - accuracy: 0.8050 - val_loss: 0.3121 - val_accuracy: 0.9067
Epoch 2/16
7/7 [==============================] - 0s 19ms/step - loss: 0.2632 - accuracy: 0.9350 - val_loss: 0.2428 - val_accuracy: 0.9381
Epoch 3/16
7/7 [==============================] - 0s 31ms/step - loss: 0.2064 - accuracy: 0.9650 - val_loss: 0.2043 - val_accuracy: 0.9574
Epoch 4/16
7/7 [===============

In [54]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.1119 - accuracy: 0.9800


[0.11188411712646484, 0.9800000190734863]

In [55]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 5ms/step - loss: 0.0811 - accuracy: 0.9825


[0.08106134086847305, 0.9825000166893005]

In [56]:
(100 - 98)/(100 - 98.25)

1.1428571428571428

# Faster Optimizers